# Asynchronous Programming in Python
Synchronous programming happens sequentially. This means code lines are executed in order from top to bottom. 

Asynchronous means tasks that take time can be awaited. This means that tasks that wait a long time can be put aside, and we can move to simultaneously trigger other tasks, while we wait.

This can help a lot with minimising the execution time of certain types of tasks like requests and parrallel tasks. However, this will not improve the preformance of tasks chains that are dependant on the output of previous tasks, as we would still have to wait for the output to arrive if we need it for the next step.

It is not multi threading, it is just switching off the processing to another function while waiting for the response.


In [2]:
import asyncio
import time

There are a few parts to this process. Asynchronous functions are not normal functions. In python they are called co-routines. To create a co-routine we use the ```async``` keyword prior to our normal function definition.
In order to run a async function (co-routine) we must create an **event loop**. An event loop tracks the execution of the parts of your programme, and transfers the processing to other parts of the code while you are waiting on specific outputs. To create an event loop we execute our asynchronous function using the ```asyncio.run()``` function.
The event loop must be told which parts of the code must run sequentially, and which parts can run asynchronously. For this inside our ```async``` functions we use the keyword ```await```.
This seems intuitive, however, we can not create single lines of code that will run asynchronously within a single coroutine. You have to create a coroutine with an await statement, and then build that into a secondary coroutine that runs the initial coroutine. Because of this, we should create as many async coroutines as necessary and then run them using a 'parent' async coroutine.

Here we have to use nest_asyncio as Jupyter notebooks require it as these types of files keep the event loop 'live'. In normal circumstances we wouldn't need that.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

async def coroutine1():
    print("Coroutine 1: Start")
    await asyncio.sleep(2)
    print("Coroutine 1: End")

async def coroutine2():
    print("Coroutine 2: Start")
    start_time = time.perf_counter()
    asyncio.create_task(coroutine1())
    end_time = time.perf_counter()
    print(f"Time taken to surpass coroutine 1: {end_time - start_time:.4f} seconds")
    print("Coroutine 2: End")
    

asyncio.run(coroutine2())


Coroutine 2: Start
Time taken to surpass coroutine 1: 0.0000 seconds
Coroutine 2: End


Coroutine 1: Start
Coroutine 1: End


🐍 Asyncio Cheat Sheet: Core Concepts
1. async def (The Definition)
Purpose: Defines a function as a coroutine.

What it does: It tells Python, "This isn't a normal function; it's a task that can pause and resume." Calling this function returns a coroutine object but does not execute the code inside immediately.

2. await (The Yield)
Purpose: Pauses the current coroutine and hands control back to the Event Loop.

What it does: It signals, "I am waiting for this specific operation to finish. Event Loop, go run other tasks in the meantime." > Note: await can only be used inside an async def function.

3. asyncio.create_task() (The Backgrounder)
Purpose: Schedules a coroutine to run immediately in the background.

What it does: It wraps a coroutine into a "Task" and adds it to the loop's queue. This allows the code to move to the next line without waiting for the task to finish, enabling concurrency.

4. asyncio.gather() (The Batcher)
Purpose: Runs multiple coroutines concurrently and waits for all of them to finish.

What it does: It is a high-level tool to manage multiple tasks. It returns a list of results once the slowest task completes.

5. asyncio.run() (The Ignition)
Purpose: The entry point that starts the engine.

What it does: It creates the Event Loop, runs the passed "main" coroutine, and shuts down the loop once finished.

In Jupyter: You often don't need asyncio.run() because Jupyter runs its own event loop. You can often just use await main_function().

So basically we create asynchronous "functions" with the keyword ```async```. We must use the ```await``` keyword to get the event loop to go to do other tasks at specific points. This in of itself is not really that useful within a single coroutine. If we think of what is going on during the coroutine, we tell the event loop to stop execution of the particular function, until that particular sentence of code is completed. This is how it normally works anyway even without async.
Where this becomes important is when we can combine multiple ```await``` statements, which has the potential to start many separate tasks, while waiting for others to be completed. To do this we must create many 'child' coroutines, and have a 'parent' coroutine to run them. In the parent we create tasks to run each child coroutine. This would be fine, except that by creating tasks we are not letting the 'parent' coroutine know what tasks to await before finishing execution. For this reason we must ```await``` each task within the parent coroutine. This can be done in a single step by using the built in ```gather()``` function. Finally we must use the ```.run()``` function to create and trigger the event loop.
How this looks in code:   

In [4]:
async def child1():
    print("Child 1: Start")
    await asyncio.sleep(3)
    print("Child 1: End")

async def child2():
    print("Child 2: Start")
    await asyncio.sleep(1)
    print("Child 2: End")   

async def child3():
    print("Child 3: Start")
    await asyncio.sleep(1)
    print("Child 3: End")

async def parent():
    print("Parent: Start")
    await asyncio.gather(
        child1(),
        child2(),
        child3()
    )
    print("Parent: End")

asyncio.run(parent())

Parent: Start
Child 1: Start
Child 2: Start
Child 3: Start
Child 2: End
Child 3: End
Child 1: End
Parent: End


As you can see from the output above, because child1 sleeps for 3 seconds, that gives time for the event loop to switch to child2, and then child3 which even though they also rest for a second each, they still finish execution before child1. 

Because we used the ```gather``` function, the parent function knows to await all of these coroutines, before closing execution on the overall parent function.

Asynchronous programming is important where we want to improve performance for tasks that have a lot of waiting, like web requests or I/O precedures. We can transfer processing to another codeblock while we await the response, making it more efficient. 

Now this has become important in GenAI. API calls to LLMs can sometimes have long wait times. We can use async to transfer to other processing tasks while we wait for the LLM to return the result

We will demonstrate this below - we will use the scenario where we have API calls to a sequence of models. The first example will be the models working sequentially


In [10]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")

client = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")
messages1 = [{"role": "user", "content": "Tell me about the standard model of quantum physics"},{"role": "system", "content": "You are a helpful assistant. Your task is to Write a basic summary of the topic, including the most important points"}]

def fetch_response(model, messages):
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response

def main():
    start_time = time.perf_counter()
    response1 = fetch_response("openai/gpt-4o-mini", messages1)
    response2 = fetch_response("google/gemini-2.0-flash-001", messages1)
    output1 = response1.choices[0].message.content
    output2 = response2.choices[0].message.content

    responses = f"Output 1: {output1}\n\nOutput 2: {output2}"
    messages2 = [{"role": "user", "content": responses}, {"role": "system", "content": "You are a helpful assistant. Your task is to Compare the two summaries and provide a single, concise summary that combines the key points from both."}]
    final_response = fetch_response(model="google/gemini-3-pro-preview", messages=messages2)
    end_time = time.perf_counter()
    final_output = final_response.choices[0].message.content
    print("Final Summary:\n", final_output)
    print(f"\nTotal Time Taken: {end_time - start_time:.4f} seconds")

main()


Final Summary:
 Here is a concise summary combining the key points of both descriptions:

The Standard Model is the prevailing theoretical framework in quantum physics that describes the fundamental constituents of matter and three of the four fundamental forces. Synthesized from principles of quantum field theory and symmetry, it functions as a "periodic table" for physics, classifying elementary particles into two main groups: **fermions** (matter building blocks, subdivided into quarks and leptons) and **bosons** (force carriers, including photons, gluons, W/Z bosons, and the mass-imparting Higgs boson).

While the model has been rigorously validated through experiments—most notably the 2012 discovery of the Higgs boson—it is considered an incomplete theory. Its major limitations include the exclusion of gravity (which is described by general relativity) and its inability to explain cosmic phenomena such as dark matter, dark energy, neutrino masses, and matter-antimatter asymmetry.


So we can see if we run the models sequentially we end up running for around 30 seconds.

Now lets create exactly the same process yet run the first model call asynchronously

In [ ]:
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")
messages1 = [{"role": "user", "content": "Tell me about the standard model of quantum physics"},{"role": "system", "content": "You are a helpful assistant. Your task is to Write a basic summary of the topic, including the most important points"}]

async def fetch_response(model, messages):
    response = await client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response

async def main():
    start_time = time.perf_counter()
    response1, response2 = await asyncio.gather(fetch_response("openai/gpt-4o-mini", messages1), fetch_response("google/gemini-2.0-flash-001", messages1)
    output1 = response1.choices[0].message.content
    output2 = response2.choices[0].message.content

    responses = f"Output 1: {output1}\n\nOutput 2: {output2}"
    messages2 = [{"role": "user", "content": responses}, {"role": "system", "content": "You are a helpful assistant. Your task is to Compare the two summaries and provide a single, concise summary that combines the key points from both."}]
    final_response = await fetch_response(model="google/gemini-3-pro-preview", messages=messages2)
    end_time = time.perf_counter()
    final_output = final_response.choices[0].message.content
    print("Final Summary:\n", final_output)
    print(f"\nTotal Time Taken: {end_time - start_time:.4f} seconds")

await main()